In [1]:
import numpy as np
import pandas as pd
import random
import torch
import matplotlib.pyplot as plt
from torch.nn.utils import clip_grad_norm_
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler 
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup

SEED = 123
BATCH_SIZE = 16
LEARNING_RATE = 2e-5
WEIGHT_DECAY = 1e-2
EPSILON = 1e-8
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)


In [ ]:
#sdfgaskdf

In [2]:
def read_imdb(data_dir, is_train):
    """读取IMDb评论数据集文本序列和标签"""
    data, labels = [], []
    df = pd.read_csv(data_dir).loc[:80000] if is_train else pd.read_csv(data_dir).loc[80000:]
    data_list = df.loc[:,'discourse_text'].values
    label_list_ = df.loc[:,'discourse_type'].values
    type_dict = {'Lead':0,'Position':1,'Claim':2,'Counterclaim':3,'Rebuttal':4,'Evidence':5,'Concluding Statement':6}
    label_list = []
    for i in label_list_:
        label_list.append(type_dict[i])
    if len(data_list)!=len(label_list):
        return 'length bug'
    else:
        n = len(data_list)
        for data_ in range(n):
            sentence_split = data_list[data_].split(".")[:-1] if data_list[data_].split('.')[-1] == str('') else data_list[data_].split(".")
            label_split = [label_list[data_] for i in range(len(sentence_split))]
            for j in range(len(sentence_split)):
                if sentence_split[j]!=' ':
                    data.append(sentence_split[j]) 
                    labels.append(label_split[j])
        return data, labels

train_data = read_imdb('./train.csv', is_train=True)
print('训练集数目：', len(train_data[0]))
for x, y in zip(train_data[0][:10], train_data[1][:10]):
    print('标签：', y, 'review:', x[0:60])

训练集数目： 191792
标签： 0 review: Modern humans today are always on their phone
标签： 0 review:  They are always on their phone more than 5 hours a day no s
标签： 0 review: All they do is text back and forward and just have group Cha
标签： 0 review:  They even do it while driving
标签： 1 review: They are some really bad consequences when stuff happens whe
标签： 5 review: Some certain areas in the United States ban phones from clas
标签： 5 review: When people have phones, they know about certain apps that t
标签： 5 review: Apps like Facebook Twitter Instagram and Snapchat
标签： 5 review:  So like if a friend moves away and you want to be in contac
标签： 5 review:  People always have different ways how to communicate with a


In [3]:
sentences = train_data[0]
labels = train_data[1]
def lower(input_list):
    output_list = []
    for i in input_list:
        output_list.append(i.lower())
    return output_list
sentences = lower(sentences)

In [4]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [5]:
max_len = 0

# For every sentence...
for sent in sentences:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode(sent, add_special_tokens=True)
    
    # Update the maximum sentence length.
    max_len = max(max_len, len(input_ids))

print('Max sentence length: ', max_len)

Max sentence length:  390


In [6]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 50,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences[0])
print('Token IDs:', input_ids[0])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
C:\ProgramData\Anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2226: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Original:  modern humans today are always on their phone
Token IDs: tensor([ 101, 2715, 4286, 2651, 2024, 2467, 2006, 2037, 3042,  102,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0])


In [7]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

172,612 training samples
19,180 validation samples


In [8]:
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=32)

test_sampler = SequentialSampler(val_dataset)
test_dataloader = DataLoader(val_dataset, sampler=test_sampler, batch_size=32)

In [9]:
for i, (train, mask, label) in enumerate(train_dataloader):
    print(train.shape, mask.shape, label.shape)               #torch.Size([16, 128]) torch.Size([16, 128]) torch.Size([16, 1])
    break
print('len(train_dataloader)=', len(train_dataloader))        #500

torch.Size([32, 50]) torch.Size([32, 50]) torch.Size([32])
len(train_dataloader)= 5395


In [10]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels = 7)     #num_labels表示2个分类，好评和差评
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [11]:
optimizer = AdamW(model.parameters(), lr = 2e-5, eps = 1e-8)

In [12]:
epochs = 10
# training steps 的数量: [number of batches] x [number of epochs]. 
total_steps = len(train_dataloader) * epochs
# 设计 learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = 0, num_training_steps = total_steps)


In [13]:
def binary_acc(preds, labels):      #preds.shape=(16, 2) labels.shape=torch.Size([16, 1])
    correct = torch.eq(torch.max(preds, dim=1)[1], labels.flatten()).float()      #eq里面的两个参数的shape=torch.Size([16])    
    acc = correct.sum().item() / len(correct)
    return acc

In [14]:
import time
import datetime
def format_time(elapsed):    
    elapsed_rounded = int(round((elapsed)))    
    return str(datetime.timedelta(seconds=elapsed_rounded))   #返回 hh:mm:ss 形式的时间

In [15]:
def train1(model, optimizer):
    t0 = time.time()
    avg_loss, avg_acc = [],[]   
    model.train()
    for step, batch in enumerate(train_dataloader):
        # 每隔40个batch 输出一下所用时间.
        if step % 40 == 0 and not step == 0:            
            elapsed = format_time(time.time() - t0)             
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        b_input_ids, b_input_mask, b_labels = batch[0].long().to(device), batch[1].long().to(device), batch[2].long().to(device)
        
        output = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        loss, logits = output[0], output[1] 
    
        avg_loss.append(loss.item())
       
        acc = binary_acc(logits, b_labels)
        avg_acc.append(acc)
        
        optimizer.zero_grad()
        loss.backward()
        clip_grad_norm_(model.parameters(), 1.0)      #大于1的梯度将其设为1.0, 以防梯度爆炸
        optimizer.step()              #更新模型参数
        scheduler.step()              #更新learning rate
        
    avg_acc = np.array(avg_acc).mean()
    avg_loss = np.array(avg_loss).mean()
    return avg_loss, avg_acc

In [16]:
train1(model,optimizer)

  Batch    40  of  5,395.    Elapsed: 0:00:06.
  Batch    80  of  5,395.    Elapsed: 0:00:11.
  Batch   120  of  5,395.    Elapsed: 0:00:17.
  Batch   160  of  5,395.    Elapsed: 0:00:23.
  Batch   200  of  5,395.    Elapsed: 0:00:28.
  Batch   240  of  5,395.    Elapsed: 0:00:34.
  Batch   280  of  5,395.    Elapsed: 0:00:39.
  Batch   320  of  5,395.    Elapsed: 0:00:45.
  Batch   360  of  5,395.    Elapsed: 0:00:51.
  Batch   400  of  5,395.    Elapsed: 0:00:56.
  Batch   440  of  5,395.    Elapsed: 0:01:02.
  Batch   480  of  5,395.    Elapsed: 0:01:08.
  Batch   520  of  5,395.    Elapsed: 0:01:13.
  Batch   560  of  5,395.    Elapsed: 0:01:19.
  Batch   600  of  5,395.    Elapsed: 0:01:25.
  Batch   640  of  5,395.    Elapsed: 0:01:30.
  Batch   680  of  5,395.    Elapsed: 0:01:35.
  Batch   720  of  5,395.    Elapsed: 0:01:39.
  Batch   760  of  5,395.    Elapsed: 0:01:44.
  Batch   800  of  5,395.    Elapsed: 0:01:49.
  Batch   840  of  5,395.    Elapsed: 0:01:54.
  Batch   880

(1.0545312783638563, 0.6277398053753476)

In [17]:
def evaluate(model):    
    avg_acc = []    
    model.eval()         #表示进入测试模式
      
    with torch.no_grad():
        for batch in test_dataloader:
            b_input_ids, b_input_mask, b_labels = batch[0].long().to(device), batch[1].long().to(device), batch[2].long().to(device)
        
            output = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
            
            acc = binary_acc(output[0], b_labels)
            avg_acc.append(acc)
    avg_acc = np.array(avg_acc).mean()
    return avg_acc

In [19]:
test_acc = evaluate(model)
print("epoch={},测试准确率={}".format(1, test_acc))

epoch=1,测试准确率=0.64578125
